In [2]:
import numpy as np
import keras

def svm_loss_and_grad(W, X, y, delta=1.0):
    """
    W: pesos (C x D)
    X: datos (D x N)
    y: etiquetas (N,)
    """
    num_classes = W.shape[0]
    num_train = X.shape[1]

    scores = W.dot(X)  # C x N
    correct_scores = scores[y, np.arange(num_train)]  # (N,)
    margins = np.maximum(0, scores - correct_scores + delta)
    margins[y, np.arange(num_train)] = 0
    loss = np.sum(margins) / num_train

    # Gradiente
    binary = margins > 0
    binary = binary.astype(float)
    row_sum = np.sum(binary, axis=0)
    binary[y, np.arange(num_train)] = -row_sum
    dW = binary.dot(X.T) / num_train  # (C x D)

    return loss, dW

# ------------------------------
# Cargar y preparar datos
# ------------------------------
(Xtr, Ytr), (_, _) = keras.datasets.cifar10.load_data()
Ytr = Ytr.flatten()

Xtr_flat = Xtr.reshape(Xtr.shape[0], -1)
Xtr_bias = np.hstack([Xtr_flat, np.ones((Xtr_flat.shape[0], 1))])  # Bias trick
Xtr_T = Xtr_bias.T  # D x N
Ytr = np.array(Ytr)

# ------------------------------
# Gradient Descent con Mini-batch
# ------------------------------
# Inicializar pesos
np.random.seed(0)
C, D = 10, Xtr_T.shape[0]
W = 0.001 * np.random.randn(C, D)

# Hiperparámetros
num_iters = 1000
batch_size = 512
learning_rate = 1e-7

loss_history = []

for it in range(num_iters):
    batch_indices = np.random.choice(Xtr_T.shape[1], batch_size, replace=False)
    X_batch = Xtr_T[:, batch_indices]  # D x batch_size
    y_batch = Ytr[batch_indices]       # batch_size

    # Calcular pérdida y gradiente
    loss, grad = svm_loss_and_grad(W, X_batch, y_batch)
    W -= learning_rate * grad  # Actualización de pesos

    loss_history.append(loss)
    if it % 100 == 0 or it == num_iters - 1:
        print(f"Iter {it}: Loss = {loss:.4f}")

Iter 0: Loss = 55.6846
Iter 100: Loss = 11.1339
Iter 200: Loss = 9.8456
Iter 300: Loss = 9.5654
Iter 400: Loss = 8.7830
Iter 500: Loss = 8.2365
Iter 600: Loss = 7.5592
Iter 700: Loss = 7.7278
Iter 800: Loss = 7.5289
Iter 900: Loss = 7.7323
Iter 999: Loss = 7.1332


In [3]:
(_, _), (Xte, Yte) = keras.datasets.cifar10.load_data()
Yte = Yte.flatten()

Xte_flat = Xte.reshape(Xte.shape[0], -1)
Xte_bias = np.hstack([Xte_flat, np.ones((Xte_flat.shape[0], 1))])  # Bias trick
Xte_T = Xte_bias.T  # D x N_test (3073 x 10000)

# Predecir usando los pesos entrenados
scores_test = W.dot(Xte_T)  # shape: (10, 10000)
Yte_pred = np.argmax(scores_test, axis=0)  # predicciones: (10000,)

# Calcular accuracy
test_accuracy = np.mean(Yte_pred == Yte)
print(f"\nTest accuracy: {test_accuracy:.4f}")


Test accuracy: 0.2739
